In [1]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('../data/real_final_train.csv')
test = pd.read_csv('../data/real_final_test.csv')

In [3]:
display(train)

,VCL_0,VCL_38,education,urban,gender,engnat,hand,religion,hand.1,religion.1,age_cat,married,orientation,familysize,ASD,nerdiness,Qs,TIPI_left,TIPI_right,VCL_1
0,2,0,2,1,3,1,2,12,2,12,2,1,4,4,2,1,2.346154,2.75,2.333333,1.0
1,1,1,4,2,2,1,1,2,1,2,4,2,1,4,2,1,2.269231,3.50,2.000000,1.0
2,1,1,2,1,1,2,1,2,1,2,4,3,2,4,2,1,2.346154,5.00,2.000000,1.0
3,2,0,1,3,1,1,2,1,2,1,1,1,1,2,2,1,2.384615,3.50,2.500000,1.0
4,2,1,1,2,2,2,2,12,2,12,1,1,1,1,2,0,2.423077,3.75,2.666667,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2,1,2,2,2,1,1,1,1,1,1,1,3,3,2,0,2.307692,3.75,2.166667,1.0
14996,2,0,4,1,2,2,1,3,1,3,4,2,1,3,2,1,2.730769,4.25,2.500000,1.0
14997,2,0,2,2,2,1,1,1,1,1,2,1,2,3,1,1,2.884615,5.00,2.000000,1.0
14998,2,2,3,2,2,1,1,12,1,12,2,2,4,2,1,0,2.615385,4.50,2.500000,1.0


In [4]:
columns = [col for col in train.columns.to_list() if col not in ['nerdiness']]

In [5]:
data = train[columns]
target = train['nerdiness']

In [24]:
def objective(trial, data=data, target=target):
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.15, random_state=777)
    param = {
        'tree_method': 'gpu_hist',
        'lambda' : trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha' : trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree' : trial.suggest_categorical('colsample_bytree', [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample' : trial.suggest_categorical('subsample', [0.4, 0.5, 0.6, 0.7, 0.8, 1.0]),
        'learning_rate' : trial.suggest_categorical('learning_rate', [0.008, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]),
        'n_estimators' : 10000,
        'max_depth' : trial.suggest_categorical('max_depth', [5, 7, 9, 11, 13, 15, 17]),
        'random_state' : trial.suggest_categorical('random_state', [777]),
        'min_child_weight' : trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)
    
    model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100, verbose=False)
    
    preds = model.predict(test_x)
    
    auc = auc(test_y, preds)
    
    return auc

In [25]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-08-02 17:05:49,573] A new study created in memory with name: no-name-8eaef033-6627-435b-90b8-c27a26603503
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:05:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[W 2022-08-02 17:05:57,112] Trial 0 failed because of the following error: UnboundLocalError("local variable 'auc' referenced before assignment")
Traceback (most recent call last):
  File "C:\Users\Home\AppData\Roaming\Python\Python38\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Home\AppData\Local\Temp/ipykernel_6120/102249080.py", line 21, in objective
    auc = auc(test_y, preds)
UnboundLocalError: local variable 'auc' referenced before assignment


UnboundLocalError: local variable 'auc' referenced before assignment

In [10]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.993496,2022-08-02 16:07:45.276459,2022-08-02 16:07:56.931427,0 days 00:00:11.654968,3.989022,0.4,0.006121,0.008,17,27,777,0.4,COMPLETE
1,1,0.992599,2022-08-02 16:07:56.932427,2022-08-02 16:08:04.701350,0 days 00:00:07.768923,2.364119,0.4,0.077136,0.010,15,48,777,0.6,COMPLETE
2,2,0.967336,2022-08-02 16:08:04.702350,2022-08-02 16:08:06.600780,0 days 00:00:01.898430,0.104313,0.6,5.675833,0.018,11,208,777,1.0,COMPLETE
3,3,0.958441,2022-08-02 16:08:06.601780,2022-08-02 16:08:11.207814,0 days 00:00:04.606034,0.087768,0.9,0.316002,0.008,7,161,777,0.7,COMPLETE
4,4,0.978599,2022-08-02 16:08:11.208814,2022-08-02 16:08:14.085022,0 days 00:00:02.876208,0.002130,0.8,0.004866,0.018,11,78,777,0.6,COMPLETE
5,5,0.943412,2022-08-02 16:08:14.086022,2022-08-02 16:08:21.166612,0 days 00:00:07.080590,0.011006,0.5,0.029392,0.008,13,151,777,0.6,COMPLETE
6,6,0.993496,2022-08-02 16:08:21.167612,2022-08-02 16:08:24.571376,0 days 00:00:03.403764,0.001883,0.8,0.001336,0.018,13,30,777,0.5,COMPLETE
7,7,0.994297,2022-08-02 16:08:24.572376,2022-08-02 16:08:28.194189,0 days 00:00:03.621813,1.008671,0.8,0.020883,0.010,15,36,777,0.8,COMPLETE
8,8,0.869142,2022-08-02 16:08:28.195189,2022-08-02 16:08:34.729658,0 days 00:00:06.534469,0.013675,0.5,0.074659,0.014,15,203,777,0.5,COMPLETE
9,9,0.911430,2022-08-02 16:08:34.730658,2022-08-02 16:08:41.433164,0 days 00:00:06.702506,0.191495,1.0,0.001879,0.014,7,144,777,0.5,COMPLETE


In [12]:
Best_trial = study.best_trial.params
Best_trial["n_estimators"], Best_trial["tree_method"] = 10000, 'gpu_hist'
Best_trial

{'lambda': 2.6015349393290106,
 'alpha': 0.029573418216154546,
 'colsample_bytree': 0.3,
 'subsample': 0.5,
 'learning_rate': 0.012,
 'max_depth': 5,
 'random_state': 777,
 'min_child_weight': 294,
 'n_estimators': 10000,
 'tree_method': 'gpu_hist'}

In [15]:
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=5, random_state=777, shuffle=True)
auc=[]
n=0
for trn_idx, test_idx in kf.split(train[columns], train['nerdiness']):
    X_tr, X_val = train[columns].iloc[trn_idx], train[columns].iloc[test_idx]
    y_tr, y_val = train['nerdiness'].iloc[trn_idx], train['nerdiness'].iloc[test_idx]
    model = xgb.XGBClassifier(**Best_trial)
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False)
    preds += model.predict(test[columns])/kf.n_splits
    auc.append(roc_auc_score(y_val, model.predict(X_val)))
    print(f'fold: {n+1} ==> auc: {auc[n]}')
    n += 1

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:11:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold: 1 ==> auc: 0.7391350072538153
[16:11:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 2 ==> auc: 0.7428133809003612
[16:12:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 3 ==> auc: 0.7244283908772333
[16:12:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 4 ==> auc: 0.7360031098914392
[16:12:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


fold: 5 ==> auc: 0.743615248950527


In [16]:
np.mean(auc)

0.7371990275746751

In [17]:
preds

array([1., 1., 1., ..., 1., 1., 1.])

In [18]:
len(preds)

35452

In [19]:
sub = pd.read_csv('../data/sample_submission.csv')

In [20]:
sub.head()

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1


In [23]:
sub['nerdiness']=preds
sub.to_csv('../data/submission.csv', index=False)

In [22]:
sub.shape

(35452, 2)